In [1]:
from gsmmutils import ModelValidator, MyModel
from cobra.flux_analysis import pfba
import gsmmutils.utils.utils
gsmmutils.DATA_PATH = "../data"
from utils import *

In [2]:
model = MyModel(r"../data/model_with_media.xml", "e_Biomass__cytop")
mv = ModelValidator(model, "ATPm__cytop")

Loading

Reactions: 4583
Metabolites: 3732
Genes: 1693
Model loaded


/home/ecunha/miniforge3/envs/model_analysis/lib/python3.10/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]


In [3]:
# mv.validate()

Check general growth rate

In [4]:
model.summary()

Running pFBA


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,6.745,0,0.00%
C00009__extr,EX_C00009__dra,0.03147,0,0.00%
C00011__extr,EX_C00011__dra,8.21,1,99.75%
C00059__extr,EX_C00059__dra,0.02092,0,0.00%
C00080__extr,EX_C00080__dra,0.05308,0,0.00%
C00120__extr,EX_C00120__dra,0.0007589,10,0.09%
C00205__extr,EX_C00205__dra,95.86,0,0.00%
C00244__extr,EX_C00244__dra,0.5173,0,0.00%
C00305__extr,EX_C00305__dra,0.0001463,0,0.00%
C00378__extr,EX_C00378__dra,0.0004362,12,0.06%


In [5]:
hetero_model = model.copy()
hetero_model.exchanges.EX_C00033__dra.bounds = (-1, 1000)
hetero_model.summary()

/home/ecunha/miniforge3/envs/model_analysis/lib/python3.10/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]


Running pFBA


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,6.888,0,0.00%
C00009__extr,EX_C00009__dra,0.03914,0,0.00%
C00011__extr,EX_C00011__dra,8.21,1,80.21%
C00033__extr,EX_C00033__dra,1,2,19.54%
C00059__extr,EX_C00059__dra,0.02601,0,0.00%
C00080__extr,EX_C00080__dra,0.06601,0,0.00%
C00120__extr,EX_C00120__dra,0.0009438,10,0.09%
C00205__extr,EX_C00205__dra,102.7,0,0.00%
C00244__extr,EX_C00244__dra,0.6433,0,0.00%
C00305__extr,EX_C00305__dra,0.0001819,0,0.00%


In [6]:
hetero_model.exchanges.EX_C00033__dra.bounds = (0, 1000)
hetero_model.exchanges.EX_C00221__dra.bounds = (-1, 1000)
hetero_model.summary()

/home/ecunha/miniforge3/envs/model_analysis/lib/python3.10/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
/home/ecunha/miniforge3/envs/model_analysis/lib/python3.10/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]


Running pFBA


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,6.745,0,0.00%
C00009__extr,EX_C00009__dra,0.03147,0,0.00%
C00011__extr,EX_C00011__dra,8.21,1,99.75%
C00059__extr,EX_C00059__dra,0.02092,0,0.00%
C00080__extr,EX_C00080__dra,0.05308,0,0.00%
C00120__extr,EX_C00120__dra,0.0007589,10,0.09%
C00205__extr,EX_C00205__dra,95.86,0,0.00%
C00244__extr,EX_C00244__dra,0.5173,0,0.00%
C00305__extr,EX_C00305__dra,0.0001463,0,0.00%
C00378__extr,EX_C00378__dra,0.0004362,12,0.06%


Check the growth rate for each condition

In [7]:
#matrix = experimental_data_processing("../data/Matriz- DCCR Dunaliella salina_dfba.xlsx", model)
#matrix.conditions = "Resume"
#print("Simulating")
#simulations(matrix, model)

In [8]:
from cobra.flux_analysis import flux_variability_analysis as fva
with model as temp_model:
    constraint = temp_model.problem.Constraint(
            temp_model.reactions.EX_C00205__dra.flux_expression + temp_model.reactions.R09542_hn437__chlo.flux_expression + 
        temp_model.reactions.R09542_hn680__chlo.flux_expression
        +temp_model.reactions.R09503_hn438__lum.flux_expression*4 + temp_model.reactions.R09503_hn673__lum.flux_expression*4,
            lb=0,
            ub=0)
    temp_model.add_cons_vars(constraint)
    constraint = temp_model.problem.Constraint(
            temp_model.reactions.R09503_hn438__lum.flux_expression + temp_model.reactions.R09503_hn673__lum.flux_expression,
            lb=0,
            ub=199.44)
    temp_model.add_cons_vars(constraint)
    sol = temp_model.optimize()
    fva_sol = fva(temp_model, ["EX_C00205__dra", "R09542_hn437__chlo", "R09542_hn680__chlo", "R09503_hn438__lum", "R09503_hn673__lum"], fraction_of_optimum=1.0)
    print(sol.status)

optimal


In [9]:
with model as tmp_model:
    tmp_model.exchanges.EX_C00244__dra.bounds = (-1,-1)
    # tmp_model.exchanges.EX_C00205__dra.bounds = (-500,-500)
    tmp_model.exchanges.EX_C00533__dra.bounds = (0,0)
    tmp_model.exchanges.EX_C00014__dra.bounds = (0,0)
    tmp_model.objective = "DM_C02094__chlo"
    sol = tmp_model.optimize()
    print(sol.status)
    print(tmp_model.summary(sol))
    # print(model.metabolites.C00533__chlo.summary(sol))

/home/ecunha/miniforge3/envs/model_analysis/lib/python3.10/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
/home/ecunha/miniforge3/envs/model_analysis/lib/python3.10/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
/home/ecunha/miniforge3/envs/model_analysis/lib/python3.10/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior

optimal
Objective
1.0 DM_C02094__chlo = 0.2052500000000002

Uptake
------
  Metabolite       Reaction  Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra 5.747         0   0.00%
C00011__extr EX_C00011__dra  8.21         1 100.00%
C00205__extr EX_C00205__dra   269         0   0.00%
C00244__extr EX_C00244__dra     1         0   0.00%
C00305__extr EX_C00305__dra 75.28         0   0.00%

Secretion
---------
     Metabolite           Reaction    Flux  C-Number  C-Flux
  C00244__cytop   DM_C00244__cytop      -1         0   0.00%
   C00305__vacu    DM_C00305__vacu  -75.28         0   0.00%
   C02094__chlo    DM_C02094__chlo -0.2053        40 100.00%
photon646__chlo DM_photon646__chlo  -71.49         0   0.00%
 photon680__lum  DM_photon680__lum   -81.7         0   0.00%
   C00007__extr     EX_C00007__dra  -11.08         0   0.00%

